<a href="https://colab.research.google.com/github/ericttran3/coingecko-crypto-web-scraper/blob/master/colab/coingecko_web_scraper_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoinGecko Web Scraper

This notebook is designed to teach you how to obtain data from top Cryptocurrency exchanges and API sources to build a trading bot for price arbitrage strategies.

- CoinGecko: https://github.com/man-c/pycoingecko



### Install Packages

In [4]:
!pip install pycoingecko --quiet

### Import Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import random
from random import randint
import time
from time import sleep
from datetime import datetime
from IPython.core.display import HTML, display
from pycoingecko import CoinGeckoAPI
from google.cloud import storage
from google.cloud import bigquery

### Configure Jupyter Notebook

In [6]:
%matplotlib inline

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_colwidth', 255)

display(HTML("<style>.container {width:85%} </style>"))

# Ingest Data

In [7]:
# Instantiate API and store in alias
cg = CoinGeckoAPI()
cg.ping()

{'gecko_says': '(V3) To the Moon!'}

### Get Coins

In [8]:
def get_coins(max_page):
    
    # Track how long python script takes to execute
    global df
    start = time.process_time()

    combined_list = []
    for page in range(1,max_page):
    
        try:
            coins_market = cg.get_coins_markets(vs_currency='usd', per_page=250, page=page, price_change_percentage='1h,24h,7d,14d,30d,200d,1y')
            combined_list += coins_market
            delay = randint(1, 2) 
            sleep(delay)
        except:
            pass
    
    # Create a dataframe from list of dictionaries
    df = pd.DataFrame(data=combined_list)
    df['created_date'] = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
    print("Execution time: {} Seconds".format(round(time.process_time() - start)))
    
    return df

In [9]:
# Maximum number of page as of 6/1/2021 is 31
get_coins(33)

Execution time: 1 Seconds


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33535.00,6.290190e+11,1.0,7.052382e+11,4.667362e+10,34123.00,31109.000000,84.100000,0.25142,-4.043289e+09,-0.63869,1.873041e+07,2.100000e+07,2.100000e+07,64805.00,-48.28640,2021-04-14T11:54:46.763Z,67.810000,49322.43202,2013-07-06T00:00:00.000Z,None,2021-06-08T23:55:17.174Z,-13.215923,0.140952,244.356226,88.192724,0.251420,-42.939572,-10.190474,2021-06-08 23:57:23
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2528.02,2.940686e+11,2.0,NaN,4.834420e+10,2636.31,2321.900000,-64.443200,-2.48579,-8.878673e+09,-2.93077,1.162131e+08,NaN,NaN,4356.99,-42.06448,2021-05-12T14:41:48.623Z,0.432979,582895.04368,2015-10-20T00:00:00.000Z,"{'times': 99.69806429391757, 'currency': 'btc', 'percentage': 9969.806429391758}",2021-06-08T23:54:24.517Z,-4.247369,0.219675,935.112804,436.373006,-2.485788,-35.390023,-6.660965,2021-06-08 23:57:23
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.00,6.261368e+10,3.0,NaN,1.064188e+11,1.02,0.989798,0.009172,0.92347,1.681763e+08,0.26932,6.246236e+10,6.246236e+10,NaN,1.32,-24.23655,2018-07-24T00:00:00.000Z,0.572521,75.08923,2015-03-02T00:00:00.000Z,None,2021-06-08T23:06:42.731Z,-0.065618,0.132882,0.264922,0.202824,0.923466,0.358276,0.488422,2021-06-08 23:57:23
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615,353.63,5.467789e+10,4.0,6.033909e+10,5.689857e+09,364.45,323.460000,-2.631283,-0.73858,-8.901188e+08,-1.60185,1.545337e+08,1.705337e+08,1.705337e+08,686.31,-48.61645,2021-05-10T07:24:17.097Z,0.039818,885558.18945,2017-10-19T00:00:00.000Z,None,2021-06-08T23:55:12.220Z,2.944500,0.432471,1934.042123,1160.873410,-0.738582,-45.178602,0.155476,2021-06-08 23:57:23
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/large/cardano.png?1547034860,1.58,5.073018e+10,5.0,7.119161e+10,5.159314e+09,1.60,1.410000,0.022911,1.47004,7.723493e+07,0.15248,3.206639e+10,4.500000e+10,4.500000e+10,2.45,-35.57970,2021-05-16T07:44:28.033Z,0.019253,8082.65083,2020-03-13T02:22:55.044Z,None,2021-06-08T23:54:43.714Z,2.479627,0.396852,1730.250560,1394.673266,1.470045,-2.145325,-9.313620,2021-06-08 23:57:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7773,taurus-chain,trt,Taurus Chain,https://assets.coingecko.com/coins/images/11069/large/TRT.jpg?1587912808,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+09,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-06-08T23:31:42.612Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 23:57:23
7774,mango-markets,mngo,Mango Markets,https://assets.coingecko.com/coins/images/14773/large/mango.png?1618369889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 23:57:23
7775,apeusd-link-synthetic-usd-dec-2021,apeUSD-LINK-DEC21,apeUSD-LINK Synthetic USD (Dec 2021),https://assets.coingecko.com/coins/images/14396/large/apeusd.jpg?1615905248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 23:57:23
7776,thinkium,tkm,Thinkium,https:/

In [14]:
df.shape

(7778, 35)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7778 entries, 0 to 7777
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        7778 non-null   object 
 1   symbol                                    7778 non-null   object 
 2   name                                      7778 non-null   object 
 3   image                                     7778 non-null   object 
 4   current_price                             7707 non-null   float64
 5   market_cap                                7710 non-null   float64
 6   market_cap_rank                           2829 non-null   float64
 7   fully_diluted_valuation                   961 non-null    float64
 8   total_volume                              7727 non-null   float64
 9   high_24h                                  6298 non-null   float64
 10  low_24h                             

In [13]:
df['coin_count'] = len(df['id'])

In [16]:
df.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date,coin_count
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33535.00,6.290190e+11,1.0,7.052382e+11,4.667362e+10,34123.00,31109.000000,84.100000,0.25142,-4.043289e+09,-0.63869,1.873041e+07,2.100000e+07,2.100000e+07,64805.00,-48.28640,2021-04-14T11:54:46.763Z,67.810000,49322.43202,2013-07-06T00:00:00.000Z,None,2021-06-08T23:55:17.174Z,-13.215923,0.140952,244.356226,88.192724,0.251420,-42.939572,-10.190474,2021-06-08 23:57:23,7778
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2528.02,2.940686e+11,2.0,NaN,4.834420e+10,2636.31,2321.900000,-64.443200,-2.48579,-8.878673e+09,-2.93077,1.162131e+08,NaN,NaN,4356.99,-42.06448,2021-05-12T14:41:48.623Z,0.432979,582895.04368,2015-10-20T00:00:00.000Z,"{'times': 99.69806429391757, 'currency': 'btc', 'percentage': 9969.806429391758}",2021-06-08T23:54:24.517Z,-4.247369,0.219675,935.112804,436.373006,-2.485788,-35.390023,-6.660965,2021-06-08 23:57:23,7778
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.00,6.261368e+10,3.0,NaN,1.064188e+11,1.02,0.989798,0.009172,0.92347,1.681763e+08,0.26932,6.246236e+10,6.246236e+10,NaN,1.32,-24.23655,2018-07-24T00:00:00.000Z,0.572521,75.08923,2015-03-02T00:00:00.000Z,None,2021-06-08T23:06:42.731Z,-0.065618,0.132882,0.264922,0.202824,0.923466,0.358276,0.488422,2021-06-08 23:57:23,7778
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615,353.63,5.467789e+10,4.0,6.033909e+10,5.689857e+09,364.45,323.460000,-2.631283,-0.73858,-8.901188e+08,-1.60185,1.545337e+08,1.705337e+08,1.705337e+08,686.31,-48.61645,2021-05-10T07:24:17.097Z,0.039818,885558.18945,2017-10-19T00:00:00.000Z,None,2021-06-08T23:55:12.220Z,2.944500,0.432471,1934.042123,1160.873410,-0.738582,-45.178602,0.155476,2021-06-08 23:57:23,7778
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/large/cardano.png?1547034860,1.58,5.073018e+10,5.0,7.119161e+10,5.159314e+09,1.60,1.410000,0.022911,1.47004,7.723493e+07,0.15248,3.206639e+10,4.500000e+10,4.500000e+10,2.45,-35.57970,2021-05-16T07:44:28.033Z,0.019253,8082.65083,2020-03-13T02:22:55.044Z,None,2021-06-08T23:54:43.714Z,2.479627,0.396852,1730.250560,1394.673266,1.470045,-2.145325,-9.313620,2021-06-08 23:57:23,7778


In [17]:
!pwd

/content


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/coingecko-crypto-list_{}.csv'.format(datetime.today().strftime("%Y-%m-%d")), index=False)

### Get Coin Details

In [21]:
columns = ['id', 'symbol', 'name', 'market_cap_rank']
# coins_df = df[df['market_cap'] > 0][columns]
coins_df = df[columns]

In [ ]:
coins = list(coins_df['id'])
coins

In [23]:
len(coins)

7778

In [24]:
# Track how long python script takes to execute
start = time.process_time()
timeout = time.time() + 3600 # 60 minutes

records = []

for coin in coins:
    coin_details = cg.get_coin_by_id(id=coin, vs_currency='usd')
    
    # Get id
    try:
        coin_id = coin_details['id']
    except:
        coin_id = ''
    
    # Get hashing algorith
    try:
        coin_algorithm = coin_details['hashing_algorithm']
    except:
        coin_algorithm = ''
        
    # Get coin categories
    try:
        coin_categories = coin_details['categories']
    except:
        coin_categories = ''
        
    # Get coin's country of origin
    try:
        coin_country_origin = coin_details['country_origin']
    except:
        coin_country_origin = ''
        
    # Get coin's gensis date
    try:
        coin_genesis_date = coin_details['genesis_date']
    except:
        coin_genesis_date = ''
        
    # Get coin's ICO start date
    try:
        coin_ico_start_date = coin_details['ico_data']['ico_start_date']
    except:
        coin_ico_start_date = ''
        
    # Get coin's ICO end date
    try:
        coin_ico_end_date = coin_details['ico_data']['ico_end_date']
    except:
        coin_ico_end_date = ''    
        
    # Get coin's short description
    try:
        coin_short_desc = coin_details['ico_data']['short_desc']
    except:
        coin_short_desc = ''

    # Get coin's full description
    try:
        coin_description = coin_details['description']['en']
    except:
        coin_description = ''
        
    # Get coins ico base sale amount
    try:
        coin_base_public_sale_amount = coin_details['ico_data']['base_public_sale_amount']
    except:
        coin_base_public_sale_amount = ''
        
    # Get coins ico quoted sale amount
    try:
        coin_quote_public_sale_amount = coin_details['ico_data']['quote_public_sale_amount']
    except:
        coin_quote_public_sale_amount = ''          
        
    # Get coin website
    try:
        coin_website = coin_details['links']['homepage']
    except:
        coin_website = ''
        
    # Get coin blockchain website
    try:
        blockchain_website = coin_details['links']['blockchain_site']
    except:
        blockchain_website = ''
        
    # Get coin forum
    try:
        coin_forum = coin_details['links']['official_forum_url']
    except:
        coin_forum = ''
        
    # Get Twitter handle
    try:
        twitter = coin_details['links']['twitter_screen_name']
    except:
        twitter = ''
        
    # Get Facebook handle
    try:
        facebook = coin_details['links']['facebook_username']
    except:
        facebook = ''
        
    # Get Reddit handle
    try:
        subreddit_url = coin_details['links']['subreddit_url']
    except:
        subreddit_url = ''
        
    # Social Media Community
    # Get twitter followers
    try:
        twitter_followers = coin_details['community_data']['twitter_followers']
    except:
        twitter_followers = ''
        
    # Get reddit average post 48 hours
    try:
        reddit_avg_post_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:
        reddit_avg_post_48h = ''
        
    # Get reddit average comments 48 hours
    try:
        reddit_avg_comments_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:        
        reddit_avg_comments_48h = ''
        
    # Get reddit subscribers
    try:
        reddit_subscribers = coin_details['community_data']['reddit_subscribers']
    except:
        reddit_subscribers = ''
        
    # Get reddit active accounts 48 hours
    try:
        reddit_active_accounts_48h = coin_details['community_data']['reddit_accounts_active_48h']
    except:
        reddit_active_accounts_48h = ''
    
    # Get Alexa rank
    try:
        alexa_rank = coin_details['public_interest_stats']['alexa_rank']
    except:
        alexa_rank = ''
        
    # Get Bing matches
    try:
        bing_matches = coin_details['public_interest_stats']['bing_matches']
    except:
        bing_matches = ''        
        
    # Get coin's sentiment data
    try:
        coin_sentiment = coin_details['sentiment_votes_up_percentage']
    except:
        coin_sentiment = ''
        
    # Get coingecko score
    try:
        coin_coingecko_score = coin_details['coingecko_score']
    except:
        coin_coingecko_score = ''
        
    # Get developer score
    try: 
        coin_developer_score = coin_details['developer_score']
    except:
        coin_developer_score = ''
        
    # Get community score
    try:
        coin_community_score = coin_details['community_score']
    except:
        coin_community_score = ''
        
    # Get liquidity score
    try:
        coin_liquidity_score = coin_details['liquidity_score']
    except:
        coin_liquidity_score = ''
        
    # Get public interest score
    try:
        coin_public_interest_score = coin_details['public_interest_score']
    except:
        coin_public_interest_score = ''
        
    
    # Developer Community
    # Get Github repos
    try:
        github_repos = coin_details['links']['repos_url']['github']
    except:
        github_repos = ''

    # Get github forks        
    try:
        github_forks = coin_details['developer_data']['forks']
    except:
        github_forks = ''
        
    # Get github stars
    try:
        github_stars = coin_details['developer_data']['stars']
    except:
        github_stars = ''
        
    # Get github subscribers
    try:
        github_subscribers = coin_details['developer_data']['subscribers']
    except:
        github_subscribers = ''
        
    # Get github issues
    try:
        github_issues = coin_details['developer_data']['total_issues']
    except:
        github_issues = ''
        
    # Get github closed issues
    try:
        github_closed_issues = coin_details['developer_data']['closed_issues']
    except:
        github_closed_issues = ''
        
    # Get github pulled requests merged
    try:
        github_pr_merged = coin_details['developer_data']['pull_requests_merged']
    except:
        github_pr_merged = ''
        
    # Get github pull requests contribuors
    try:
        github_pr_contributers = coin_details['developer_data']['pull_request_contributors']
    except:
        github_pr_contributers = ''
        
    # Get github developer additions
    try:
        github_additions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['additions']
    except:
        github_additions_4w = ''
        
    # Get github developer deletions
    try:
        github_deletions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['deletions']
    except:
        github_deletions_4w = ''
        
    # Get github commits in last 4 weeks
    try:
        github_commits_4w = coin_details['developer_data']['commit_count_4_weeks']
    except:
        github_commits_4w = ''
    
    record = (coin_id, coin_algorithm, coin_categories, coin_country_origin, coin_genesis_date, coin_ico_start_date, coin_ico_end_date, 
              coin_short_desc, coin_description, coin_base_public_sale_amount, coin_quote_public_sale_amount, 
              coin_website, blockchain_website, coin_forum, twitter, facebook, twitter_followers, subreddit_url, reddit_avg_post_48h, reddit_avg_comments_48h, 
              reddit_subscribers, reddit_active_accounts_48h, alexa_rank, bing_matches, coin_sentiment, coin_coingecko_score, coin_developer_score,
              coin_community_score, coin_liquidity_score, coin_public_interest_score, github_repos, github_forks, github_stars, github_subscribers, 
              github_issues, github_closed_issues, github_pr_merged, github_pr_contributers, github_additions_4w, github_deletions_4w, github_commits_4w)

    records.append(record)
    delay = randint(1, 2) 
    sleep(delay) # Delay pagination by 1 to 2 seconds each time

print("Execution time: {} minutes".format(round(time.process_time() - start,2)))

Execution time: 94.92 minutes


In [32]:
columns = ['coin_id', 'coin_algorithm', 'coin_categories', 'coin_country_origin', 'coin_genesis_date', 'coin_ico_start_date', 'coin_ico_end_date', 'coin_short_desc', 
           'coin_base_public_sale_amount', 'coin_quote_public_sale_amount', 'coin_website', 'blockchain_website', 'coin_forum', 'twitter', 'facebook', 'twitter_followers',
           'subreddit_url', 'reddit_avg_post_48h', 'reddit_avg_comments_48h', 'reddit_subscribers', 'reddit_active_accounts_48h', 'alexa_rank', 'bing_matches',
           'coin_sentiment', 'coin_coingecko_score', 'coin_developer_score', 'coin_community_score', 'coin_liquidity_score', 'coin_public_interest_score', 'github_repos', 
           'github_forks', 'github_stars', 'github_subscribers', 'github_issues', 'github_closed_issues', 'github_pr_merged', 'github_pr_contributers', 
           'github_additions_4w', 'github_deletions_4w', 'github_commits_4w']
df_cd = pd.DataFrame(data=records, columns=columns)
df_cd

,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026,https://www.reddit.com/r/Bitcoin/,7.167,7.167,3056135,9564,9440.0,None,53.55,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996,54527,3816,6274,5692,8678,729,5220.0,-5205.0,381
1,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648,https://www.reddit.com/r/ethereum,7.000,7.000,999488,2731,8793.0,None,64.24,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276,30935,2108,5618,5423,3766,531,9935.0,-4824.0,58
2,tether,None,"[USD Stablecoin, Stablecoins]",,None,,,,,,"[https://tether.to/, , ]","[https://blockchair.com/bitcoin/omni/property/31, https://www.omniexplorer.info/asset/31, https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7, https://ethplorer.io/address/0xdac17f958d2ee523a2206206994597c13d831ec7, https://tronscan....","[, , ]",Tether_to,tether.to,122013,None,0.000,0.000,0,0,74251.0,None,69.87,42.054,0.000,10.771,107.145,0.066,[],0,0,0,0,0,0,0,NaN,NaN,0
3,binancecoin,None,"[Centralized Exchange Token (CEX), Binance Smart Chain Ecosystem, Exchange-based Tokens]",,2017-07-08,,,,,,"[https://www.binance.com/, , ]","[https://binance.mintscan.io/, https://explorer.binance.org/, https://bscscan.com, https://etherscan.io/token/0xB8c77482e45F1F44dE1745F52C74426C631bDD52, https://ethplorer.io/address/0xB8c77482e45F1F44dE1745F52C74426C631bDD52]","[, , ]",binance,binanceexchange,4185392,https://www.reddit.com/r/binance,9.818,9.818,480846,3325,888.0,None,65.24,68.888,73.255,64.872,86.742,14.394,"[https://github.com/binance-exchange/binance-official-api-docs, https://github.com/binance-exchange/node-binance-api, https://github.com/binance-exchange/php-binance-api]",2020,3405,498,122,122,55,26,0.0,0.0,0
4,cardano,None,[Smart Contract Platform],,None,,,,,,"[https://www.cardano.org/en/home/, , ]","[https://cardanoexplorer.com/, https://cardanoscan.io/, https://blockchair.com/cardano, https://adaex.org/, https://adastat.net/]","[, , ]",CardanoStiftung,,498080,https://www.reddit.com/r/cardano,6.273,6.273,509936,2231,42419.0,None,72.82,65.752,70.463,59.135,84.797,0.302,"[https://github.com/input-output-hk/cardano-sl, https://github.com/input-output-hk/js-cardano-wasm, https://github.com/input-output-hk/plutus, https://github.com/input-output-hk/rust-cardano, https://github.com/input-output-hk/cardano-chain]",608,3620,445,850,665,1706,80,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [34]:
# Save to local
df_cd.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/coingecko-crypto-coin-details_{}.csv'.format(datetime.today().strftime("%Y-%m-%d")), index=False)

### Combine Crypto Coins and Coin Details DataFrames

In [35]:
df

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date,coin_count
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33535.00,6.290190e+11,1.0,7.052382e+11,4.667362e+10,34123.00,31109.000000,84.100000,0.25142,-4.043289e+09,-0.63869,1.873041e+07,2.100000e+07,2.100000e+07,64805.00,-48.28640,2021-04-14T11:54:46.763Z,67.810000,49322.43202,2013-07-06T00:00:00.000Z,None,2021-06-08T23:55:17.174Z,-13.215923,0.140952,244.356226,88.192724,0.251420,-42.939572,-10.190474,2021-06-08 23:57:23,7778
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2528.02,2.940686e+11,2.0,NaN,4.834420e+10,2636.31,2321.900000,-64.443200,-2.48579,-8.878673e+09,-2.93077,1.162131e+08,NaN,NaN,4356.99,-42.06448,2021-05-12T14:41:48.623Z,0.432979,582895.04368,2015-10-20T00:00:00.000Z,"{'times': 99.69806429391757, 'currency': 'btc', 'percentage': 9969.806429391758}",2021-06-08T23:54:24.517Z,-4.247369,0.219675,935.112804,436.373006,-2.485788,-35.390023,-6.660965,2021-06-08 23:57:23,7778
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.00,6.261368e+10,3.0,NaN,1.064188e+11,1.02,0.989798,0.009172,0.92347,1.681763e+08,0.26932,6.246236e+10,6.246236e+10,NaN,1.32,-24.23655,2018-07-24T00:00:00.000Z,0.572521,75.08923,2015-03-02T00:00:00.000Z,None,2021-06-08T23:06:42.731Z,-0.065618,0.132882,0.264922,0.202824,0.923466,0.358276,0.488422,2021-06-08 23:57:23,7778
3,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/large/binance-coin-logo.png?1547034615,353.63,5.467789e+10,4.0,6.033909e+10,5.689857e+09,364.45,323.460000,-2.631283,-0.73858,-8.901188e+08,-1.60185,1.545337e+08,1.705337e+08,1.705337e+08,686.31,-48.61645,2021-05-10T07:24:17.097Z,0.039818,885558.18945,2017-10-19T00:00:00.000Z,None,2021-06-08T23:55:12.220Z,2.944500,0.432471,1934.042123,1160.873410,-0.738582,-45.178602,0.155476,2021-06-08 23:57:23,7778
4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/large/cardano.png?1547034860,1.58,5.073018e+10,5.0,7.119161e+10,5.159314e+09,1.60,1.410000,0.022911,1.47004,7.723493e+07,0.15248,3.206639e+10,4.500000e+10,4.500000e+10,2.45,-35.57970,2021-05-16T07:44:28.033Z,0.019253,8082.65083,2020-03-13T02:22:55.044Z,None,2021-06-08T23:54:43.714Z,2.479627,0.396852,1730.250560,1394.673266,1.470045,-2.145325,-9.313620,2021-06-08 23:57:23,7778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7773,taurus-chain,trt,Taurus Chain,https://assets.coingecko.com/coins/images/11069/large/TRT.jpg?1587912808,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+09,NaN,NaN,0.00000,None,NaN,0.00000,None,None,2021-06-08T23:31:42.612Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 23:57:23,7778
7774,mango-markets,mngo,Mango Markets,https://assets.coingecko.com/coins/images/14773/large/mango.png?1618369889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-08 23:57:23,7778
7775,apeusd-link-synthetic-usd-dec-2021,apeUSD-LINK-DEC21,apeUSD-LINK Synthetic USD (Dec 2021),https://assets.coingecko.com/coins/images/14396/large/apeusd.jpg?1615905248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021

In [36]:
df_cd

,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026,https://www.reddit.com/r/Bitcoin/,7.167,7.167,3056135,9564,9440.0,None,53.55,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996,54527,3816,6274,5692,8678,729,5220.0,-5205.0,381
1,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648,https://www.reddit.com/r/ethereum,7.000,7.000,999488,2731,8793.0,None,64.24,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276,30935,2108,5618,5423,3766,531,9935.0,-4824.0,58
2,tether,None,"[USD Stablecoin, Stablecoins]",,None,,,,,,"[https://tether.to/, , ]","[https://blockchair.com/bitcoin/omni/property/31, https://www.omniexplorer.info/asset/31, https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7, https://ethplorer.io/address/0xdac17f958d2ee523a2206206994597c13d831ec7, https://tronscan....","[, , ]",Tether_to,tether.to,122013,None,0.000,0.000,0,0,74251.0,None,69.87,42.054,0.000,10.771,107.145,0.066,[],0,0,0,0,0,0,0,NaN,NaN,0
3,binancecoin,None,"[Centralized Exchange Token (CEX), Binance Smart Chain Ecosystem, Exchange-based Tokens]",,2017-07-08,,,,,,"[https://www.binance.com/, , ]","[https://binance.mintscan.io/, https://explorer.binance.org/, https://bscscan.com, https://etherscan.io/token/0xB8c77482e45F1F44dE1745F52C74426C631bDD52, https://ethplorer.io/address/0xB8c77482e45F1F44dE1745F52C74426C631bDD52]","[, , ]",binance,binanceexchange,4185392,https://www.reddit.com/r/binance,9.818,9.818,480846,3325,888.0,None,65.24,68.888,73.255,64.872,86.742,14.394,"[https://github.com/binance-exchange/binance-official-api-docs, https://github.com/binance-exchange/node-binance-api, https://github.com/binance-exchange/php-binance-api]",2020,3405,498,122,122,55,26,0.0,0.0,0
4,cardano,None,[Smart Contract Platform],,None,,,,,,"[https://www.cardano.org/en/home/, , ]","[https://cardanoexplorer.com/, https://cardanoscan.io/, https://blockchair.com/cardano, https://adaex.org/, https://adastat.net/]","[, , ]",CardanoStiftung,,498080,https://www.reddit.com/r/cardano,6.273,6.273,509936,2231,42419.0,None,72.82,65.752,70.463,59.135,84.797,0.302,"[https://github.com/input-output-hk/cardano-sl, https://github.com/input-output-hk/js-cardano-wasm, https://github.com/input-output-hk/plutus, https://github.com/input-output-hk/rust-cardano, https://github.com/input-output-hk/cardano-chain]",608,3620,445,850,665,1706,80,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [37]:
result = pd.merge(df, df_cd, how="left", left_on='id', right_on='coin_id')
result

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,price_change_percentage_14d_in_currency,price_change_percentage_1h_in_currency,price_change_percentage_1y_in_currency,price_change_percentage_200d_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_30d_in_currency,price_change_percentage_7d_in_currency,created_date,coin_count,coin_id,coin_algorithm,coin_categories,coin_country_origin,coin_genesis_date,coin_ico_start_date,coin_ico_end_date,coin_short_desc,coin_base_public_sale_amount,coin_quote_public_sale_amount,coin_website,blockchain_website,coin_forum,twitter,facebook,twitter_followers,subreddit_url,reddit_avg_post_48h,reddit_avg_comments_48h,reddit_subscribers,reddit_active_accounts_48h,alexa_rank,bing_matches,coin_sentiment,coin_coingecko_score,coin_developer_score,coin_community_score,coin_liquidity_score,coin_public_interest_score,github_repos,github_forks,github_stars,github_subscribers,github_issues,github_closed_issues,github_pr_merged,github_pr_contributers,github_additions_4w,github_deletions_4w,github_commits_4w
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579,33535.00,6.290190e+11,1.0,7.052382e+11,4.667362e+10,34123.00,31109.000000,84.100000,0.25142,-4.043289e+09,-0.63869,1.873041e+07,2.100000e+07,2.100000e+07,64805.00,-48.28640,2021-04-14T11:54:46.763Z,67.810000,49322.43202,2013-07-06T00:00:00.000Z,None,2021-06-08T23:55:17.174Z,-13.215923,0.140952,244.356226,88.192724,0.251420,-42.939572,-10.190474,2021-06-08 23:57:23,7778,bitcoin,SHA-256,[Cryptocurrency],,2009-01-03,,,,,,"[http://www.bitcoin.org, , ]","[https://blockchair.com/bitcoin/, https://btc.com/, https://btc.tokenview.com/, , ]","[https://bitcointalk.org/, , ]",bitcoin,bitcoins,2682026,https://www.reddit.com/r/Bitcoin/,7.167,7.167,3056135,9564,9440.0,None,53.55,81.317,98.883,75.318,100.038,0.381,"[https://github.com/bitcoin/bitcoin, https://github.com/bitcoin/bips]",28996,54527,3816,6274,5692,8678,729,5220.0,-5205.0,381
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/large/ethereum.png?1595348880,2528.02,2.940686e+11,2.0,NaN,4.834420e+10,2636.31,2321.900000,-64.443200,-2.48579,-8.878673e+09,-2.93077,1.162131e+08,NaN,NaN,4356.99,-42.06448,2021-05-12T14:41:48.623Z,0.432979,582895.04368,2015-10-20T00:00:00.000Z,"{'times': 99.69806429391757, 'currency': 'btc', 'percentage': 9969.806429391758}",2021-06-08T23:54:24.517Z,-4.247369,0.219675,935.112804,436.373006,-2.485788,-35.390023,-6.660965,2021-06-08 23:57:23,7778,ethereum,Ethash,[Smart Contract Platform],,2015-07-30,2014-07-20T00:00:00.000Z,2014-09-01T00:00:00.000Z,A decentralized platform for applications,1,0.00074794,"[https://www.ethereum.org/, , ]","[https://etherscan.io/, https://ethplorer.io/, https://blockchair.com/ethereum, https://eth.tokenview.com/, https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd]","[https://forum.ethereum.org/, , ]",ethereum,ethereumproject,1346648,https://www.reddit.com/r/ethereum,7.000,7.000,999488,2731,8793.0,None,64.24,78.268,97.236,65.118,100.782,0.473,"[https://github.com/ethereum/go-ethereum, https://github.com/ethereum/py-evm, https://github.com/ethereum/aleth, https://github.com/ethereum/web3.py, https://github.com/ethereum/solidity, https://github.com/ethereum/sharding, https://github.com/ethere...",11276,30935,2108,5618,5423,3766,531,9935.0,-4824.0,58
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/large/Tether-logo.png?1598003707,1.00,6.261368e+10,3.0,NaN,1.064188e+11,1.02,0.989798,0.009172,0.92347,1.681763e+08,0.26932,6.246236e+10,6.246236e+10,NaN,1.32,-24.23655,2018-07-24T00:00:00.000Z,0.572521,75.08923,2015-03-02T00:00:00.000Z,None,2021-06-08T2

In [38]:
# Save to local
result.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/coingecko-crypto-coin-details-full_{}.csv'.format(datetime.today().strftime("%Y-%m-%d")), index=False)

# Explore API

In [ ]:
# Get current data (name, price, market, ... including exchange tickers) for a coin
coin_details = cg.get_coin_by_id(id='ethereum', vs_currency='usd')
coin_details

In [ ]:
type(coin_details)

dict

In [ ]:
coin_details.keys()

dict_keys(['id', 'symbol', 'name', 'asset_platform_id', 'platforms', 'block_time_in_minutes', 'hashing_algorithm', 'categories', 'public_notice', 'additional_notices', 'localization', 'description', 'links', 'image', 'country_origin', 'genesis_date', 'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage', 'ico_data', 'market_cap_rank', 'coingecko_rank', 'coingecko_score', 'developer_score', 'community_score', 'liquidity_score', 'public_interest_score', 'market_data', 'community_data', 'developer_data', 'public_interest_stats', 'status_updates', 'last_updated', 'tickers'])

In [ ]:
coin_details['id']


'ethereum'

In [39]:
coin_details['description']['en']

'Solrise Finance is a decentralized fund management and investment protocol built on Solana - the fastest blockchain to date.'

In [ ]:
coin_details['platforms']

{'': '',
 'binance-smart-chain': '0x2170ed0880ac9a755fd29b2688956bd959f933f8',
 'huobi-token': '0x64ff637fb478863b7468bc97d30a5bf3a428a1fd'}

In [ ]:
coin_details['block_time_in_minutes']

0

In [ ]:
coin_details['hashing_algorithm']

'Ethash'

In [ ]:
coin_details['categories']

['Smart Contract Platform']

In [ ]:
coin_details['links']

{'announcement_url': ['', ''],
 'bitcointalk_thread_identifier': 428589,
 'blockchain_site': ['https://etherscan.io/',
  'https://ethplorer.io/',
  'https://blockchair.com/ethereum',
  'https://eth.tokenview.com/',
  'https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd'],
 'chat_url': ['', '', ''],
 'facebook_username': 'ethereumproject',
 'homepage': ['https://www.ethereum.org/', '', ''],
 'official_forum_url': ['https://forum.ethereum.org/', '', ''],
 'repos_url': {'bitbucket': [],
  'github': ['https://github.com/ethereum/go-ethereum',
   'https://github.com/ethereum/py-evm',
   'https://github.com/ethereum/aleth',
   'https://github.com/ethereum/web3.py',
   'https://github.com/ethereum/solidity',
   'https://github.com/ethereum/sharding',
   'https://github.com/ethereum/casper',
   'https://github.com/paritytech/parity']},
 'subreddit_url': 'https://www.reddit.com/r/ethereum',
 'telegram_channel_identifier': '',
 'twitter_screen_name': 'ethereum'}

In [ ]:
coin_details['links']['homepage']

['https://www.ethereum.org/', '', '']

In [ ]:
coin_details['links']['blockchain_site']

['https://etherscan.io/',
 'https://ethplorer.io/',
 'https://blockchair.com/ethereum',
 'https://eth.tokenview.com/',
 'https://hecoinfo.com/token/0x64ff637fb478863b7468bc97d30a5bf3a428a1fd']

In [ ]:
coin_details['links']['official_forum_url']

['https://forum.ethereum.org/', '', '']

In [ ]:
coin_details['links']['twitter_screen_name']

'ethereum'

In [ ]:
coin_details['links']['facebook_username']

'ethereumproject'

In [ ]:
coin_details['links']['subreddit_url']

'https://www.reddit.com/r/ethereum'

In [ ]:
coin_details['links']['repos_url']['github']

['https://github.com/ethereum/go-ethereum',
 'https://github.com/ethereum/py-evm',
 'https://github.com/ethereum/aleth',
 'https://github.com/ethereum/web3.py',
 'https://github.com/ethereum/solidity',
 'https://github.com/ethereum/sharding',
 'https://github.com/ethereum/casper',
 'https://github.com/paritytech/parity']

In [ ]:
coin_details['country_origin']

''

In [ ]:
coin_details['genesis_date']

'2015-07-30'

In [ ]:
coin_details['sentiment_votes_up_percentage']

64.2

In [ ]:
coin_details['sentiment_votes_down_percentage']

35.8

In [ ]:
coin_details['ico_data']

{'accepting_currencies': '',
 'amount_for_sale': None,
 'base_pre_sale_amount': None,
 'base_public_sale_amount': 1.0,
 'bounty_detail_url': '',
 'country_origin': '',
 'description': None,
 'hardcap_amount': None,
 'hardcap_currency': '',
 'ico_end_date': '2014-09-01T00:00:00.000Z',
 'ico_start_date': '2014-07-20T00:00:00.000Z',
 'kyc_required': True,
 'links': {},
 'pre_sale_available': None,
 'pre_sale_end_date': None,
 'pre_sale_ended': False,
 'pre_sale_start_date': None,
 'quote_pre_sale_amount': None,
 'quote_pre_sale_currency': '',
 'quote_public_sale_amount': 0.00074794,
 'quote_public_sale_currency': 'BTC',
 'short_desc': 'A decentralized platform for applications',
 'softcap_amount': None,
 'softcap_currency': '',
 'total_raised': None,
 'total_raised_currency': '',
 'whitelist_available': None,
 'whitelist_end_date': None,
 'whitelist_start_date': None,
 'whitelist_url': ''}

In [ ]:
coin_details['ico_data']['ico_start_date']

'2014-07-20T00:00:00.000Z'

In [ ]:
coin_details['ico_data']['ico_end_date']

'2014-09-01T00:00:00.000Z'

In [ ]:
coin_details['ico_data']['short_desc']

'A decentralized platform for applications'

In [ ]:
coin_details['ico_data']['base_public_sale_amount']

1.0

In [ ]:
coin_details['ico_data']['quote_public_sale_amount']

0.00074794

In [ ]:
coin_details['coingecko_score']

78.268

In [ ]:
coin_details['developer_score']

97.236

In [ ]:
coin_details['community_score']

65.118

In [ ]:
coin_details['liquidity_score']

100.782

In [ ]:
coin_details['public_interest_score']

0.473

In [ ]:
coin_details['community_data']

In [ ]:
coin_details['community_data']['facebook_likes']

In [ ]:
coin_details['community_data']['twitter_followers']

1346648

In [ ]:
coin_details['community_data']['reddit_average_posts_48h']

7.0

In [ ]:
coin_details['community_data']['reddit_average_comments_48h']

313.75

In [ ]:
coin_details['community_data']['reddit_subscribers']

999488

In [ ]:
coin_details['community_data']['reddit_accounts_active_48h']

2731

In [ ]:
coin_details['community_data']['telegram_channel_user_count']

In [ ]:
coin_details['developer_data']['forks']

11276

In [ ]:
coin_details['developer_data']['stars']

30935

In [ ]:
coin_details['developer_data']['subscribers']

2108

In [ ]:
coin_details['developer_data']['total_issues']

5618

In [ ]:
coin_details['developer_data']['closed_issues']

5423

In [ ]:
coin_details['developer_data']['pull_requests_merged']

3766

In [ ]:
coin_details['developer_data']['pull_request_contributors']

531

In [ ]:
coin_details['developer_data']['code_additions_deletions_4_weeks']['additions']

9935

In [ ]:
coin_details['developer_data']['code_additions_deletions_4_weeks']['deletions']

-4824

In [ ]:
coin_details['developer_data']['commit_count_4_weeks']

58

In [ ]:
coin_details['public_interest_stats']['alexa_rank']

8793

In [ ]:
coin_details['public_interest_stats']['bing_matches']

# Putting it all Together

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import random
from random import randint
import time
from time import sleep
from datetime import datetime
from IPython.core.display import HTML, display
from pycoingecko import CoinGeckoAPI
from google.cloud import storage
from google.cloud import bigquery

In [ ]:

# Track how long python script takes to execute
start = time.process_time()
timeout = time.time() + 3600 # 60 minutes

# Instantiate API and store in alias
cg = CoinGeckoAPI()
cg.ping()

def get_coins(max_page):
    """
    This function takes in the number of pages to scrape cryptocurrency coin data from CoinGecko's API 
    and returns the final output into pandas dataframe.
    """

    global df
    
    # Track how long python script takes to execute
    start = time.process_time()
    combined_list = []

    for page in range(1,max_page):
    
        try:
            coins_market = cg.get_coins_markets(vs_currency='usd', per_page=250, page=page, price_change_percentage='1h,24h,7d,14d,30d,200d,1y')
            combined_list += coins_market
            delay = randint(1, 2) 
            sleep(delay)
        except:
            pass
    
    # Create a dataframe from list of dictionaries
    df = pd.DataFrame(data=combined_list)
    df['created_date'] = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
    print("Execution time: {} Seconds".format(round(time.process_time() - start)))

    return df


def get_record(coin):
    """
    This function takes json payload with details about a cryptocurreny coin and extracts the relevent 
    information and returns a unique record to be inserted into a pandas dataframe.
    """

    # Get id
    try:
        coin_id = coin_details['id']
    except:
        coin_id = ''
    
    # Get hashing algorith
    try:
        coin_algorithm = coin_details['hashing_algorithm']
    except:
        coin_algorithm = ''
        
    # Get coin categories
    try:
        coin_categories = coin_details['categories']
    except:
        coin_categories = ''
        
    # Get coin's country of origin
    try:
        coin_country_origin = coin_details['country_origin']
    except:
        coin_country_origin = ''
        
    # Get coin's gensis date
    try:
        coin_genesis_date = coin_details['genesis_date']
    except:
        coin_genesis_date = ''
        
    # Get coin's ICO start date
    try:
        coin_ico_start_date = coin_details['ico_data']['ico_start_date']
    except:
        coin_ico_start_date = ''
        
    # Get coin's ICO end date
    try:
        coin_ico_end_date = coin_details['ico_data']['ico_end_date']
    except:
        coin_ico_end_date = ''    
        
    # Get coin's short description
    try:
        coin_short_desc = coin_details['ico_data']['short_desc']
    except:
        coin_short_desc = ''
        
    # Get coins ico base sale amount
    try:
        coin_base_public_sale_amount = coin_details['ico_data']['base_public_sale_amount']
    except:
        coin_base_public_sale_amount = ''
        
    # Get coins ico quoted sale amount
    try:
        coin_quote_public_sale_amount = coin_details['ico_data']['quote_public_sale_amount']
    except:
        coin_quote_public_sale_amount = ''          
        
    # Get coin website
    try:
        coin_website = coin_details['links']['homepage']
    except:
        coin_website = ''
        
    # Get coin blockchain website
    try:
        blockchain_website = coin_details['links']['blockchain_site']
    except:
        blockchain_website = ''
        
    # Get coin forum
    try:
        coin_forum = coin_details['links']['official_forum_url']
    except:
        coin_forum = ''
        
    # Get Twitter handle
    try:
        twitter = coin_details['links']['twitter_screen_name']
    except:
        twitter = ''
        
    # Get Facebook handle
    try:
        facebook = coin_details['links']['facebook_username']
    except:
        facebook = ''
        
    # Get Reddit handle
    try:
        subreddit_url = coin_details['links']['subreddit_url']
    except:
        subreddit_url = ''
        
    # Social Media Community
    # Get twitter followers
    try:
        twitter_followers = coin_details['community_data']['twitter_followers']
    except:
        twitter_followers = ''
        
    # Get reddit average post 48 hours
    try:
        reddit_avg_post_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:
        reddit_avg_post_48h = ''
        
    # Get reddit average comments 48 hours
    try:
        reddit_avg_comments_48h = coin_details['community_data']['reddit_average_posts_48h']
    except:        
        reddit_avg_comments_48h = ''
        
    # Get reddit subscribers
    try:
        reddit_subscribers = coin_details['community_data']['reddit_subscribers']
    except:
        reddit_subscribers = ''
        
    # Get reddit active accounts 48 hours
    try:
        reddit_active_accounts_48h = coin_details['community_data']['reddit_accounts_active_48h']
    except:
        reddit_active_accounts_48h = ''
    
    # Get Alexa rank
    try:
        alexa_rank = coin_details['public_interest_stats']['alexa_rank']
    except:
        alexa_rank = ''
        
    # Get Bing matches
    try:
        bing_matches = coin_details['public_interest_stats']['bing_matches']
    except:
        bing_matches = ''        
        
    # Get coin's sentiment data
    try:
        coin_sentiment = coin_details['sentiment_votes_up_percentage']
    except:
        coin_sentiment = ''
        
    # Get coingecko score
    try:
        coin_coingecko_score = coin_details['coingecko_score']
    except:
        coin_coingecko_score = ''
        
    # Get developer score
    try: 
        coin_developer_score = coin_details['developer_score']
    except:
        coin_developer_score = ''
        
    # Get community score
    try:
        coin_community_score = coin_details['community_score']
    except:
        coin_community_score = ''
        
    # Get liquidity score
    try:
        coin_liquidity_score = coin_details['liquidity_score']
    except:
        coin_liquidity_score = ''
        
    # Get public interest score
    try:
        coin_public_interest_score = coin_details['public_interest_score']
    except:
        coin_public_interest_score = ''
        
    
    # Developer Community
    # Get Github repos
    try:
        github_repos = coin_details['links']['repos_url']['github']
    except:
        github_repos = ''

    # Get github forks        
    try:
        github_forks = coin_details['developer_data']['forks']
    except:
        github_forks = ''
        
    # Get github stars
    try:
        github_stars = coin_details['developer_data']['stars']
    except:
        github_stars = ''
        
    # Get github subscribers
    try:
        github_subscribers = coin_details['developer_data']['subscribers']
    except:
        github_subscribers = ''
        
    # Get github issues
    try:
        github_issues = coin_details['developer_data']['total_issues']
    except:
        github_issues = ''
        
    # Get github closed issues
    try:
        github_closed_issues = coin_details['developer_data']['closed_issues']
    except:
        github_closed_issues = ''
        
    # Get github pulled requests merged
    try:
        github_pr_merged = coin_details['developer_data']['pull_requests_merged']
    except:
        github_pr_merged = ''
        
    # Get github pull requests contribuors
    try:
        github_pr_contributers = coin_details['developer_data']['pull_request_contributors']
    except:
        github_pr_contributers = ''
        
    # Get github developer additions
    try:
        github_additions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['additions']
    except:
        github_additions_4w = ''
        
    # Get github developer deletions
    try:
        github_deletions_4w = coin_details['developer_data']['code_additions_deletions_4_weeks']['deletions']
    except:
        github_deletions_4w = ''
        
    # Get github commits in last 4 weeks
    try:
        github_commits_4w = coin_details['developer_data']['commit_count_4_weeks']
    except:
        github_commits_4w = ''
    
    record = (coin_id, coin_algorithm, coin_categories, coin_country_origin, 
              coin_genesis_date, coin_ico_start_date, coin_ico_end_date, coin_short_desc, coin_base_public_sale_amount, coin_quote_public_sale_amount, 
              coin_website, blockchain_website, coin_forum, twitter, facebook, twitter_followers, subreddit_url, reddit_avg_post_48h, reddit_avg_comments_48h, 
              reddit_subscribers, reddit_active_accounts_48h, alexa_rank, bing_matches, coin_sentiment, coin_coingecko_score, coin_developer_score,
              coin_community_score, coin_liquidity_score, coin_public_interest_score, github_repos, github_forks, github_stars, github_subscribers, 
              github_issues, github_closed_issues, github_pr_merged, github_pr_contributers, github_additions_4w, github_deletions_4w, github_commits_4w)
    
    return record

def get_coin_details(coins):
    """
    This function takes in a list of coins as an input and extracts details about cryptocurrencies
    and returns the final output in a pandas dataframe.
    """
    
    for coin in coins:
        try:
            print(coin)
            coin_details = cg.get_coin_by_id(id=coin, vs_currency='usd')
            record = get_record(coin_details)
            records.append(record)
            delay = randint(1, 2) 
            sleep(delay) # Delay pagination by 1 to 2 seconds each time
        except:
            pass

    columns = ['coin_id', 'coin_algorithm', 'coin_categories', 'coin_country_origin', 'coin_genesis_date', 'coin_ico_start_date', 'coin_ico_end_date', 'coin_short_desc', 
              'coin_base_public_sale_amount', 'coin_quote_public_sale_amount', 'coin_website', 'blockchain_website', 'coin_forum', 'twitter', 'facebook', 'twitter_followers',
              'subreddit_url', 'reddit_avg_post_48h', 'reddit_avg_comments_48h', 'reddit_subscribers', 'reddit_active_accounts_48h', 'alexa_rank', 
              'coin_sentiment', 'coin_coingecko_score', 'coin_developer_score', 'coin_community_score', 'coin_liquidity_score', 'coin_public_interest_score', 'github_repos', 
              'github_forks', 'github_stars', 'github_subscribers', 'github_issues', 'github_closed_issues', 'github_pr_merged', 'github_pr_contributers', 
              'github_additions_4w', 'github_deletions_4w', 'github_commits_4w']

    df_cd = pd.DataFrame(data=records, columns=columns)

    return df_cd

print("Execution time: {} minutes".format(round(time.process_time() - start,2)))

# Parking Lot

In [ ]:
# Get historical market data including price, market cap, and 24h volume (granularity auto)
cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='usd', days=365)

In [ ]:
# Get coin's OHLC (beta)
cg.get_coin_ohlc_by_id(id='ethereum', vs_currency='usd', days=365)

### Get Exchanges

In [25]:
# List all exchanges
exchanges = cg.get_exchanges_list()

columns = ['id', 'name', 'year_established', 'country', 'description', 'url', 'image', 'has_trading_incentive', 'trust_score', 'trust_score_rank', 'trade_volume_24h_btc', 
           'trade_volume_24h_btc_normalized']
exchanges_df = pd.DataFrame(data=exchanges, columns=columns)
exchanges_df

,id,name,year_established,country,description,url,image,has_trading_incentive,trust_score,trust_score_rank,trade_volume_24h_btc,trade_volume_24h_btc_normalized
0,binance,Binance,2017.0,Cayman Islands,,https://www.binance.com/,https://assets.coingecko.com/markets/images/52/small/binance.jpg?1519353250,False,10,1,890808.602094,890808.602094
1,gdax,Coinbase Exchange,2012.0,United States,,https://www.coinbase.com,https://assets.coingecko.com/markets/images/23/small/Coinbase_Coin_Primary.png?1621471875,False,10,2,134245.819993,134245.819993
2,kraken,Kraken,2011.0,United States,,https://r.kraken.com/Q1m9x,https://assets.coingecko.com/markets/images/29/small/kraken.jpg?1584251255,False,10,3,58053.498779,58053.498779
3,binance_us,Binance US,2019.0,United States,,https://www.binance.us/en,https://assets.coingecko.com/markets/images/469/small/Binance.png?1568875842,False,10,4,31211.538676,31211.538676
4,bitfinex,Bitfinex,2014.0,British Virgin Islands,,https://www.bitfinex.com,https://assets.coingecko.com/markets/images/4/small/BItfinex.png?1615895883,False,10,5,31396.408494,31396.408494
5,latoken,LATOKEN,2017.0,Cayman Islands,,https://latoken.com/,https://assets.coingecko.com/markets/images/124/small/LA_token.png?1605773251,False,10,6,12315.644341,12315.644341
6,gemini,Gemini,2014.0,United States,,https://gemini.sjv.io/bZ49k,https://assets.coingecko.com/markets/images/50/small/gemini.png?1605704107,False,10,7,10735.566591,10735.566591
7,crypto_com,Crypto.com,2019.0,Cayman Islands,"Crypto.com Exchange is the best place to trade crypto, with deep liquidity, low fees and best execution prices, users can trade major cryptocurrencies like Bitcoin, Ethereum, and many more and receive great CRO-powered rewards",https://crypto.com/exchange,https://assets.coingecko.com/markets/images/589/small/crypto_com.jpg?1602499898,False,10,8,7419.174991,7419.174991
8,huobi,Huobi Global,2013.0,Seychelles,,https://www.huobi.com,https://assets.coingecko.com/markets/images/25/small/1481589873352_.pic_hd.jpg?1589962155,False,9,9,254130.394721,254130.394721
9,okex,OKEx,2013.0,Belize,,https://www.okex.com,https://assets.coingecko.com/markets/images/96/small/okEX.jpg?1519349636,False,9,10,187738.370804,187738.370804


In [ ]:
# List all supported markets id and name (no pagination required)
exchange_names = cg.get_exchanges_id_name_list()

exchange_names_df = pd.DataFrame(exchange_names, columns=['id', 'name'])
exchange_names_df

### Get Trending

In [40]:
trending = cg.get_search_trending()

keys = ['id', 'coin_id', 'name', 'symbol', 'market_cap_rank', 'thumb', 'small', 'large', 'slug', 'price_btc', 'score']

trending['coins']

[{'item': {'coin_id': 9568,
   'id': 'kusama',
   'large': 'https://assets.coingecko.com/coins/images/9568/large/m4zRhP5e_400x400.jpg?1576190080',
   'market_cap_rank': 30,
   'name': 'Kusama',
   'price_btc': 0.012348655274838475,
   'score': 0,
   'slug': 'kusama',
   'small': 'https://assets.coingecko.com/coins/images/9568/small/m4zRhP5e_400x400.jpg?1576190080',
   'symbol': 'KSM',
   'thumb': 'https://assets.coingecko.com/coins/images/9568/thumb/m4zRhP5e_400x400.jpg?1576190080'}},
 {'item': {'coin_id': 15810,
   'id': 'gitcoin',
   'large': 'https://assets.coingecko.com/coins/images/15810/large/gitcoin.png?1621992929',
   'market_cap_rank': 248,
   'name': 'Gitcoin',
   'price_btc': 0.00030205523993223307,
   'score': 1,
   'slug': 'gitcoin',
   'small': 'https://assets.coingecko.com/coins/images/15810/small/gitcoin.png?1621992929',
   'symbol': 'GTC',
   'thumb': 'https://assets.coingecko.com/coins/images/15810/thumb/gitcoin.png?1621992929'}},
 {'item': {'coin_id': 4713,
   'id': 